# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [21]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
import torch.multiprocessing

from collections import OrderedDict

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [14]:
data_dir = 'flowers'

In [15]:
# TODO: Define your transforms for the training, validation, and testing sets
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)
    ]),
    
    'valid': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)
    ]),
    
    'test': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(norm_mean, norm_std)
    ]),
}


image_datasets = {x: datasets.ImageFolder(data_dir + '/' + x, data_transforms[x]) \
                  for x in ['train', 'valid', 'test']}

batch_sizes = {'train': 64, 'valid': 32, 'test': 20}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=batch_sizes[x], 
                                              shuffle=True) \
               for x in ['train', 'valid', 'test']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}

In [16]:
# shows an image from the tensor
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array(norm_mean)
    std = np.array(norm_std)
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [17]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [18]:
#images, labels = next(iter(dataloaders['train']))
#imshow(images[0], title=cat_to_name[str(labels.tolist()[0])]);

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

### GPU / CPU Management

In [47]:
torch.cuda.empty_cache()

In [48]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    #torch.cuda.empty_cache()
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

Quadro M1000M
Memory Usage:
Allocated: 0.1 GB
Cached:    0.1 GB


## Build and Train Network

### Build Network

In [49]:
# TODO: Build and train your network

## load a pre-trained netnwork
model = models.densenet121(pretrained=True)
#model = models.densenet121(pretrained=True)

# freeze feature parameters of pretrain network
for param in model.parameters():
    param.requires_grad = False

# rebuild classifier 
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(in_features = 2**10, out_features = 2**9)),
    ('relu1', nn.ReLU()),
    ('do1', nn.Dropout(p=0.2)),
    ('fc2', nn.Linear(in_features = 2**9, out_features = 2**8)),
    ('relu3', nn.ReLU()),
    ('do2', nn.Dropout(p=0.2)),
    ('fc3', nn.Linear(in_features = 2**8, out_features = 102)),
    ('output', nn.LogSoftmax(dim=1)),
]))

model.classifier = classifier

criterion = nn.NLLLoss()

learning_rate = 0.1
#optimizer = optim.Adam(model.classifier.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)
scheduler = StepLR(optimizer, step_size=3, gamma=0.5)


### Train Network

In [50]:
device

device(type='cuda')

In [51]:
epochs = 12
eval_every = 5


# move model to device GPU/CPU
model.to(device)

for epoch in range(epochs):
    running_loss = 0
    steps = 0
    
    learn_rt = scheduler.get_lr()
  
    # enumerate training dataset
    for inputs, labels in dataloaders['train']:
        steps += 1
        # move data tensors to device
        inputs_train, labels_train = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  # zero out the gradients
        
        logps = model.forward(inputs_train) # make forward pass on input
        loss = criterion(logps, labels_train) # calculate loss
        loss.backward()  # back propogate the loss
        optimizer.step() # update the weights
        
        
        # update the running loss
        running_loss += loss.item()
        
        # evalute every x steps
        if steps % eval_every == 0:
            loss_valid, accuracy = 0, 0
            model.eval()
            model.to(device)
            with torch.no_grad():
                # enumerate validation dataset
                for inputs_valid, labels_valid in dataloaders['valid']:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    
                    output_valid = model.forward(inputs_valid)
                    batch_loss = criterion(output_valid, labels_valid)
                    
                    loss_valid += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(output_valid)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels_valid.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Images {steps * batch_sizes['train'] }/{dataset_sizes['train']}.. "
                  f"Train loss: {running_loss/dataset_sizes['train']:.4f}.. "
                  f"Test loss: {loss_valid/dataset_sizes['valid']:.4f}.. "
                  f"Test accuracy: {accuracy/dataset_sizes['valid']:.4f}.. "
                  f"LR: {learn_rt}")
            
            running_loss = 0
            model.train()
    scheduler.step()

Epoch 1/12.. Images 320/6552.. Train loss: 0.0035.. Test loss: 0.1443.. Test accuracy: 0.0013.. LR: [0.1]
Epoch 1/12.. Images 640/6552.. Train loss: 0.0035.. Test loss: 0.1428.. Test accuracy: 0.0005.. LR: [0.1]
Epoch 1/12.. Images 960/6552.. Train loss: 0.0034.. Test loss: 0.1396.. Test accuracy: 0.0023.. LR: [0.1]
Epoch 1/12.. Images 1280/6552.. Train loss: 0.0034.. Test loss: 0.1351.. Test accuracy: 0.0030.. LR: [0.1]
Epoch 1/12.. Images 1600/6552.. Train loss: 0.0033.. Test loss: 0.1324.. Test accuracy: 0.0020.. LR: [0.1]
Epoch 1/12.. Images 1920/6552.. Train loss: 0.0031.. Test loss: 0.1216.. Test accuracy: 0.0058.. LR: [0.1]
Epoch 1/12.. Images 2240/6552.. Train loss: 0.0029.. Test loss: 0.1144.. Test accuracy: 0.0063.. LR: [0.1]
Epoch 1/12.. Images 2560/6552.. Train loss: 0.0029.. Test loss: 0.1045.. Test accuracy: 0.0071.. LR: [0.1]
Epoch 1/12.. Images 2880/6552.. Train loss: 0.0025.. Test loss: 0.1211.. Test accuracy: 0.0045.. LR: [0.1]
Epoch 1/12.. Images 3200/6552.. Train lo

Epoch 4/12.. Images 5440/6552.. Train loss: 0.0014.. Test loss: 0.0400.. Test accuracy: 0.0210.. LR: [0.025]
Epoch 4/12.. Images 5760/6552.. Train loss: 0.0012.. Test loss: 0.0419.. Test accuracy: 0.0212.. LR: [0.025]
Epoch 4/12.. Images 6080/6552.. Train loss: 0.0014.. Test loss: 0.0408.. Test accuracy: 0.0214.. LR: [0.025]
Epoch 4/12.. Images 6400/6552.. Train loss: 0.0013.. Test loss: 0.0412.. Test accuracy: 0.0212.. LR: [0.025]
Epoch 5/12.. Images 320/6552.. Train loss: 0.0015.. Test loss: 0.0417.. Test accuracy: 0.0206.. LR: [0.05]
Epoch 5/12.. Images 640/6552.. Train loss: 0.0014.. Test loss: 0.0419.. Test accuracy: 0.0207.. LR: [0.05]
Epoch 5/12.. Images 960/6552.. Train loss: 0.0013.. Test loss: 0.0404.. Test accuracy: 0.0211.. LR: [0.05]
Epoch 5/12.. Images 1280/6552.. Train loss: 0.0012.. Test loss: 0.0422.. Test accuracy: 0.0209.. LR: [0.05]
Epoch 5/12.. Images 1600/6552.. Train loss: 0.0011.. Test loss: 0.0419.. Test accuracy: 0.0206.. LR: [0.05]
Epoch 5/12.. Images 1920/65

Epoch 8/12.. Images 3840/6552.. Train loss: 0.0009.. Test loss: 0.0306.. Test accuracy: 0.0240.. LR: [0.025]
Epoch 8/12.. Images 4160/6552.. Train loss: 0.0009.. Test loss: 0.0273.. Test accuracy: 0.0252.. LR: [0.025]
Epoch 8/12.. Images 4480/6552.. Train loss: 0.0008.. Test loss: 0.0272.. Test accuracy: 0.0247.. LR: [0.025]
Epoch 8/12.. Images 4800/6552.. Train loss: 0.0011.. Test loss: 0.0263.. Test accuracy: 0.0249.. LR: [0.025]
Epoch 8/12.. Images 5120/6552.. Train loss: 0.0009.. Test loss: 0.0290.. Test accuracy: 0.0245.. LR: [0.025]
Epoch 8/12.. Images 5440/6552.. Train loss: 0.0008.. Test loss: 0.0257.. Test accuracy: 0.0252.. LR: [0.025]
Epoch 8/12.. Images 5760/6552.. Train loss: 0.0009.. Test loss: 0.0291.. Test accuracy: 0.0241.. LR: [0.025]
Epoch 8/12.. Images 6080/6552.. Train loss: 0.0009.. Test loss: 0.0266.. Test accuracy: 0.0248.. LR: [0.025]
Epoch 8/12.. Images 6400/6552.. Train loss: 0.0009.. Test loss: 0.0288.. Test accuracy: 0.0247.. LR: [0.025]
Epoch 9/12.. Images

Epoch 12/12.. Images 1920/6552.. Train loss: 0.0008.. Test loss: 0.0245.. Test accuracy: 0.0255.. LR: [0.0125]
Epoch 12/12.. Images 2240/6552.. Train loss: 0.0008.. Test loss: 0.0239.. Test accuracy: 0.0254.. LR: [0.0125]
Epoch 12/12.. Images 2560/6552.. Train loss: 0.0008.. Test loss: 0.0247.. Test accuracy: 0.0249.. LR: [0.0125]
Epoch 12/12.. Images 2880/6552.. Train loss: 0.0008.. Test loss: 0.0235.. Test accuracy: 0.0257.. LR: [0.0125]
Epoch 12/12.. Images 3200/6552.. Train loss: 0.0008.. Test loss: 0.0232.. Test accuracy: 0.0260.. LR: [0.0125]
Epoch 12/12.. Images 3520/6552.. Train loss: 0.0007.. Test loss: 0.0242.. Test accuracy: 0.0253.. LR: [0.0125]
Epoch 12/12.. Images 3840/6552.. Train loss: 0.0008.. Test loss: 0.0223.. Test accuracy: 0.0265.. LR: [0.0125]
Epoch 12/12.. Images 4160/6552.. Train loss: 0.0007.. Test loss: 0.0243.. Test accuracy: 0.0253.. LR: [0.0125]
Epoch 12/12.. Images 4480/6552.. Train loss: 0.0007.. Test loss: 0.0241.. Test accuracy: 0.0257.. LR: [0.0125]
E

In [52]:
torch.cuda.empty_cache()

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes